<a href="https://colab.research.google.com/github/Communist02/BIN-22-1_Mazur_Denis_ML/blob/master/DZ4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 66.7 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import math

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')


class Processing:
    def tokenize(self, text: str) -> list[str]:
        return word_tokenize(text)

    def lematize(self, tokens: list[str]) -> list[str]:
        morph3 = MorphAnalyzer()
        lemmatized_words = [morph3.parse(
            word)[0].normal_form for word in tokens]
        return lemmatized_words

    def stemming(self, tokens: list[str]) -> list[str]:
        stemmer = SnowballStemmer("russian")
        lemmatized_words = [stemmer.stem(word) for word in tokens]
        return lemmatized_words

    def vectorize(self, tokens: list[str]) -> list[int]:
        dict_vectors = {}
        result = []
        for word in tokens:
            if word in dict_vectors.keys():
                result.append(dict_vectors[word])
            else:
                dict_vectors[word] = len(dict_vectors)
                result.append(dict_vectors[word])
        return result

    def vectorize_dict(self, tokens: list[str]) -> list[int]:
        dict_vectors = {}
        result = []
        for word in tokens:
            if word not in dict_vectors.keys():
                dict_vectors[word] = len(dict_vectors)
        return dict_vectors

    def delete_stop_words(self, tokens: list[str]) -> list[int]:
        stop_words = set(stopwords.words('russian')).union(['.', ',', ':', '?', '!'])
        return [word for word in tokens if word not in stop_words]

    def bag_of_words(self, tokens: list[str]) -> dict[str]:
        dict_words = {}
        for word in tokens:
            dict_words[word] = dict_words.setdefault(word, 0) + 1
        return dict_words

    def tf(self, tokens: list[str]) -> dict[str]:
        dict_words = self.bag_of_words(tokens)
        for word in dict_words:
            dict_words[word] /= len(tokens)
        return dict_words

    def idf(self, texts: list[list[str]]) -> dict[str]:
        dict_words = {}
        big_text = []
        for text in texts:
            big_text += list(set(text))
        for word in set(big_text):
            dict_words[word] = math.log(len(texts) / big_text.count(word))
        return dict_words

    def tf_idf(self, texts: list[list[str]], indexText: int) -> dict[str]:
        tf = self.tf(texts[indexText])
        idf = self.idf(texts)
        dict_words = {}
        for word in tf:
            dict_words[word] = tf[word] * idf[word]
        return dict_words


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
class SimpleTransformer:
    def __init__(self, vocab_size, embed_dim, hidden_dim, lr=0.01):
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.lr = lr

        # Параметры модели
        self.embeddings = np.random.randn(vocab_size, embed_dim) * 0.01

        self.W_q = np.random.randn(embed_dim, hidden_dim) * 0.01
        self.W_k = np.random.randn(embed_dim, hidden_dim) * 0.01
        self.W_v = np.random.randn(embed_dim, hidden_dim) * 0.01

        self.W_out = np.random.randn(hidden_dim, vocab_size) * 0.01

    def softmax(self, x):
        exp = np.exp(x - np.max(x))
        return exp / np.sum(exp)

    def forward(self, input_indices):
        self.input_indices = input_indices
        self.x = self.embeddings[input_indices]  # shape: (seq_len, embed_dim)

        self.Q = self.x @ self.W_q  # (seq_len, hidden_dim)
        self.K = self.x @ self.W_k
        self.V = self.x @ self.W_v

        self.attn_scores = self.Q @ self.K.T / np.sqrt(self.embed_dim)  # (seq_len, seq_len)
        self.attn_weights = self.softmax(self.attn_scores)  # (seq_len, seq_len)

        self.attended = self.attn_weights @ self.V  # (seq_len, hidden_dim)
        self.context = np.mean(self.attended, axis=0)  # (hidden_dim,)

        self.logits = self.context @ self.W_out  # (vocab_size,)
        self.probs = self.softmax(self.logits)

        return self.probs

    def backward(self, target_index):
        # dL/dlogits = probs - y_one_hot
        dlogits = self.probs.copy()
        dlogits[target_index] -= 1  # (vocab_size,)

        # dL/dW_out
        dW_out = np.outer(self.context, dlogits)  # (hidden_dim, vocab_size)

        # dL/dcontext
        dcontext = dlogits @ self.W_out.T  # (hidden_dim,)

        # dcontext -> dattended (т.к. mean)
        dattended = np.ones_like(self.attended) * dcontext / self.attended.shape[0]  # (seq_len, hidden_dim)

        dV = self.attn_weights.T @ dattended  # (seq_len, hidden_dim)
        d_attn_weights = dattended @ self.V.T  # (seq_len, seq_len)

        d_scores = d_attn_weights * self.attn_weights * (1 - self.attn_weights)  # (seq_len, seq_len)

        dQ = d_scores @ self.K / np.sqrt(self.embed_dim)
        dK = d_scores.T @ self.Q / np.sqrt(self.embed_dim)

        # dW_q, dW_k, dW_v
        dW_q = self.x.T @ dQ
        dW_k = self.x.T @ dK
        dW_v = self.x.T @ dV

        # dEmbeddings
        dx_q = dQ @ self.W_q.T
        dx_k = dK @ self.W_k.T
        dx_v = dV @ self.W_v.T
        dx = dx_q + dx_k + dx_v  # (seq_len, embed_dim)

        # Градиенты по эмбеддингам
        for i, idx in enumerate(self.input_indices):
            self.embeddings[idx] -= self.lr * dx[i]

        # Обновление параметров
        self.W_out -= self.lr * dW_out
        self.W_q -= self.lr * dW_q
        self.W_k -= self.lr * dW_k
        self.W_v -= self.lr * dW_v

    def train_step(self, input_indices, target_index):
        self.forward(input_indices)
        self.backward(target_index)

    def predict(self, input_indices):
        probs = self.forward(input_indices)
        return np.argmax(probs)

In [33]:
import numpy as np

pc = Processing()
text = 'Звезды отражались в её глазах. Раньше, еще месяц назад их не было видно в черте города. Свет фонарей и смог заглушали их слабое мерцание. Все изменилось. И одним из немногих плюсов сложившейся ситуации было мерцание звезд в её глазах, и воздух, кажется стал чище. У всех у нас когда то была работа, и был дом. У некоторых были дети. У Лены была дочка. Она работала барменшой, а по вечерам подрабатывала в "клубе знакомств". Попросту говоря - была проституткой. Теперь ей уже не приходится ездить по незнакомым клиентам, каждый раз перед дверью квартиры креститься, и молиться, что бы все прошло как надо. Это тоже плюс. Но теперь у неё нет дочки. Она потерялась в первые дни, как только все это начиналось. Лена была "на вызове", когда исчезло электричество. Никто еще не знал, что это серьезно. Мобильная связь не работала, город погрузился во тьму за окнами однакомнатной квартиры, в которой возбужденный мужчина кончал в презерватив, а Лена считала секунды до очередного вызова. Она не могла как обычно принять душ, и вызвать такси, и после осознания этого, просто начала одеваться. Белье по привычке было сложено одной кучкой рядом с кроватью. Мужчина, имя которого она не захотела запоминать сказал ей спасибо и открыл дверь, что то проворчав напоследок на "долбанных электриков"... Лене было очень приятно выйти на свежий воздух, после пропахшей перегаром комнатушки. Она шла по темным улицам города, шла на "базу" пешком, и эта непроглядная тьма вокруг для неё сейчас была отражением внутреннего состояния, и поэтому она наслаждалась этой прогулкой. Она еще не знала, что электричество и водоснабжение уже не восстановят. Она не могла подумать, что через три часа её пятилетняя дочка, испугавшись темноты и одиночества, выйдет из квартиры, и пропадет навсегда. Она еще не знала, что её поиски будут бесполезны и опасны... Она просто шла по улице.'

orders = [
    'Звезды отражались',
    'Воздух стал',
    'Она раньше шла',
    'Дочка испугалась',
    'Город погрузился',
    'Она не могла месяц',
    'Электричество исчезло',
    'Поиск видно'
]

result = [
    'глазах',
    'чище',
    'темным',
    'одиночества',
    'тьму',
    'связь',
    'связь',
    'опасен'
]

tokens = pc.delete_stop_words(pc.lematize(pc.tokenize(text)))

# Векторизация словаря
vocab = list(set(tokens))
word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}

# Пример данных
X = []
y = []

# Задача - предсказать последнее слово
for phrase, label in zip(orders, result):
    tokens = pc.delete_stop_words(pc.lematize(pc.tokenize(phrase)))
    if len(tokens) < 2: continue
    input_indices = [word_to_index.get(w, 0) for w in tokens if w in word_to_index]
    label_index = word_to_index.get(label, 0)
    X.append(input_indices)
    y.append(label_index)

# Обучение
model = SimpleTransformer(vocab_size=len(vocab), embed_dim=16, hidden_dim=32)

for epoch in range(1000):
    total_loss = 0
    for i in range(len(X)):
        model.train_step(X[i], y[i])
        probs = model.forward(X[i])
        total_loss += -np.log(probs[y[i]] + 1e-9)
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss / len(X)}")

# Предсказания
print("Predictions:")
for phrase in orders:
    tokens = pc.delete_stop_words(pc.lematize(pc.tokenize(phrase)))
    input_indices = [word_to_index.get(w, 0) for w in tokens if w in word_to_index]
    pred_index = model.predict(input_indices)
    print(f"{phrase} -> {index_to_word[pred_index]}")


Epoch 0, Loss: 4.948759121512369
Predictions:
Звезды отражались -> потеряться
Воздух стал -> комнатушка
Она раньше шла -> улица
Дочка испугалась -> очень
Город погрузился -> потеряться
Она не могла месяц -> связь
Электричество исчезло -> связь
Поиск видно -> потеряться
